# ETL (Extract Transform Load)
- En esta ocasion se realizara un <b>E.T.L</b> de dos archivos `.csv` a los cuales se le aplicara un modelo de `Machine Learning` para crear un 
<i>Sistema de Recomendacion <i>.

## Importar las librerias a utilizar

In [80]:
import pandas as pd
import numpy as np
import ast
from unidecode import unidecode
import re

### Lectura de los archivos `.csv`.

In [81]:
pd.set_option('display.max_columns', None)
credits = pd.read_csv('../datasets/credits.csv')
movies = pd.read_csv('../datasets/movies_dataset.csv')

C:\Users\Martin\AppData\Local\Temp\ipykernel_19896\3179120926.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('../datasets/movies_dataset.csv')


## DF movies - ETL
* <p>Se realiza una copia del dataframe (df) como medida de no afectar al original<p>

In [82]:
movies_copia = movies.copy()

* Observación de valores nulos

In [83]:
movies_copia.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

<p style="color:rgb(190,190,190)">Se puede observar varias columnas con gran cantidad de valores nulos, por ende, podrian ser eliminadas. A su vez, seran eliminadas aquellas columnas que no aporten valor<p>

* Eliminacion de columnas

In [84]:
delete_columns = ["video","imdb_id","adult","original_title","poster_path","homepage"]

movies_copia.drop(columns=delete_columns,inplace=True)

# Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

* Eliminamos posibles duplicados del campo titulo y id para evitar peliculas repetidas

In [85]:
movies_copia.drop_duplicates(subset=["title"],inplace=True)
movies_copia.drop_duplicates(subset=["title"],inplace=True)

* Eliminacion de signos de puntuacion

In [86]:
movies_copia["title"] = movies_copia["title"].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))


* Normalizamos el titulo a minuscula

In [87]:
movies_copia["title"] = movies_copia["title"].str.lower()

* Rellenamos valores nulos por 0 en los campos de las columnas <i>revenue</i> y <i>budget</i> 

In [88]:
movies_copia.fillna(value={"budget":0,"revenue":0},inplace=True)
print(movies_copia[["revenue","budget"]].isna().sum())

revenue    0
budget     0
dtype: int64


* Eliminamos valores nulos del campo <i>release</i> con el fin de cambiar de cambiar el tipo de dato a `datetime`

In [89]:
# print(movies_copia["release_date"].info()) #dtype: object
movies_copia["release_date"] = pd.to_datetime(movies_copia["release_date"],format="%Y-%m-%d",errors="coerce").dropna()
movies_copia = movies_copia.dropna(subset="release_date")
# print(movies_copia["release_date"].info()) #dtype: datetime

print(movies_copia["release_date"].isna().sum())

0


* Se crea una columna especifica para el año de lanzamiento

In [90]:
movies_copia["release_year"]=movies_copia["release_date"].dropna().dt.year
movies_copia["release_year"] = movies_copia["release_year"].astype(int)

* Se cambia a dato `int` los valores de la columna <i>id</i> con el fin de luego realizar un merge con los id del Dataset <i>credits</i>
<p style="color:gray">Primero se realiza cambios en registros extraños<p>

In [91]:
movies_copia["id"]=movies_copia["id"].str.replace("-","") # Este paso se realiza ya que se encontraron id's con estructura de fecha ej: 2020-03-08
movies_copia["id"] = movies_copia["id"].astype(int)

* Cambio de tipo de dato de popularity.



In [92]:
movies_copia["popularity"] = movies_copia["popularity"].astype(float)

## Creamos la funcion para desanidar  los campos con diccionarios 

1) DataFrame-movies

In [93]:
# Utilizado para Listas de diccionarios, su return es una lista
def divide_columns(valores,indice, key=False,value=False):
    
    list_of_dicts = ast.literal_eval(valores) #Toma un string como input y lo evalua como una estructura de python. ej: de string ---> lista.
    new_values = []
    if(len(list_of_dicts)==0): return np.nan
    for i in range(len(list_of_dicts)):
        if(value and list_of_dicts[i][key]==value):
            new_values.append(list_of_dicts[i][indice])
        if key==False:
            new_values.append(list_of_dicts[i][indice])

    return list(set(new_values))



In [94]:
# Utilizado para Listas de diccionarios, su return es un string
def divide_list(valores,indice, key=False,value=False):
    
    # cleaned_string = valores.replace(" ", "")
    list_of_dicts = ast.literal_eval(valores) #Toma un string como input y lo evalua como una estructura de python. ej: de string ---> lista.
    if(len(list_of_dicts)==0): return "Unknown"
    lista = [list_of_dicts[i][indice] for i in range(len(list_of_dicts)) ]
        # name_genre.append(list_of_dicts[i]["name"])
    return " ".join(lista)

In [95]:
# En caso de que el input sea solo llaves de un diccionario {} y no una lista de diccionarios [{}]
def divide_dicts(valores,key):
    list_of_dicts = ast.literal_eval(valores)
    if(isinstance(list_of_dicts,dict)):
        return list_of_dicts[key]
    return valores

In [96]:
# Desanidamos belong_to_collection
movies_copia.fillna({"belongs_to_collection":"[]"},inplace=True) # Ya que la funcion divide_dicts reconoce estructuras de python, debemos realizar una falsa lista
collection = movies_copia["belongs_to_collection"].apply(lambda x: divide_dicts(x,"name"))
movies_copia["belongs_to_collection"] = movies_copia["belongs_to_collection"].replace("[]","Unknown")

In [97]:
movies_copia.sample()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year
23482,Unknown,0,"[{'id': 18, 'name': 'Drama'}]",173908,en,Quickie-mart employee Melissa and paraplegic R...,9.471018,"[{'name': 'Original Media', 'id': 2292}, {'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",2013-04-20,0.0,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,sunlight jr,5.1,25.0,2013


In [98]:
#Desanidamos genres en name
name_genre = movies_copia["genres"].apply(lambda x: divide_columns(x,"name"))

In [99]:
# Desanidamos production_countries
countries = movies_copia["production_countries"].apply(lambda x: divide_list(x,"name"))

In [100]:
#Desanidamos production_companies
movies_copia.fillna({"production_companies":"Unknown"},inplace=True)
movies_copia["production_companies"]=movies_copia["production_companies"].replace("False","[]")
company = movies_copia["production_companies"].apply(lambda x: divide_list(x,"name"))

In [101]:
# Desanidamos spoken_language
language = movies_copia["spoken_languages"].apply(lambda x: divide_list(x,"name"))

**Realizamos el renombre de las columnas modificadas**

In [102]:
name_genre= pd.DataFrame(name_genre).rename(columns={"genres":"name_genre"})
collection= pd.DataFrame(collection).rename(columns={"belongs_to_collection":"collection"})
company= pd.DataFrame(company).rename(columns={"production_companies":"company"})
countries= pd.DataFrame(countries).rename(columns={"production_countries":"countries"})
language= pd.DataFrame(language).rename(columns={"spoken_languages":"language"})

In [103]:
movies_copia.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,toy story,7.7,5415.0,1995
1,Unknown,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,jumanji,6.9,2413.0,1995
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,grumpier old men,6.5,92.0,1995
3,Unknown,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,waiting to exhale,6.1,34.0,1995
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,father of the bride part ii,5.7,173.0,1995


* Concatenamos las nuevas columnas creadas

In [104]:
movies_copia = pd.concat([movies_copia,name_genre,collection,company,countries,language],axis=1)

* Se crea la columna Retorno de Inversion (R.O.I.)

In [105]:
# Cheaquear el tipo de datos de ambas columnas
movies_copia[["revenue","budget"]].info() #revenue --> float64 #budget -->  object
# Cambio el tipo de dato de budget
movies_copia["budget"]=movies_copia["budget"].astype("Float64")

returns = np.where(movies_copia["budget"]==0,np.nan,movies_copia["revenue"]/movies_copia["budget"])
movies_copia["return"] = returns
movies_copia.fillna({"return":0},inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 42196 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   revenue  42196 non-null  float64
 1   budget   42196 non-null  object 
dtypes: float64(1), object(1)
memory usage: 989.0+ KB


* Por ultimo limpiamos el Dataset

In [106]:
movies_copia = movies_copia.drop(columns=["belongs_to_collection","genres","production_companies","production_countries","spoken_languages"])

In [107]:
movies_copia.head(2)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,release_year,name_genre,collection,company,countries,language,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,toy story,7.7,5415.0,1995,"[Comedy, Family, Animation]",Toy Story Collection,Pixar Animation Studios,United States of America,English,12.451801
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,jumanji,6.9,2413.0,1995,"[Fantasy, Family, Adventure]",[],TriStar Pictures Teitler Film Interscope Commu...,United States of America,English Français,4.043035


## DF credits - ETL
* <p>Se realiza una copia del dataframe (df) como medida de no afectar al original<p>

In [108]:
credits_copy = credits.copy()
credits_copy.shape

(45476, 3)

* Se extrae el nombre de los actores que han participado en la pelicula

In [109]:
credits_copy["actors_names"] = credits_copy["cast"].apply(lambda x: divide_columns(x,"name"))

* Se extrae el nombre de los directores que han participado en la pelicula

In [110]:
credits_copy["director_names"]=credits_copy["crew"].apply(lambda x: divide_columns(x,"name","job","Director"))

### Realizo un merge entre <i>credits_copy</i> y <i>movies_copy</i> utilizando la columna `id`

In [111]:
cast_and_crew = credits_copy[["id","actors_names","director_names"]]

movies_merged = movies_copia.merge(cast_and_crew,on="id",how="inner") #Merge utilizando el id


In [112]:
movies_merged.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,release_year,name_genre,collection,company,countries,language,return,actors_names,director_names
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,toy story,7.7,5415.0,1995,"[Comedy, Family, Animation]",Toy Story Collection,Pixar Animation Studios,United States of America,English,12.451801,"[Laurie Metcalf, Jim Varney, Wallace Shawn, R....",[John Lasseter]
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,jumanji,6.9,2413.0,1995,"[Fantasy, Family, Adventure]",[],TriStar Pictures Teitler Film Interscope Commu...,United States of America,English Français,4.043035,"[Adam Hann-Byrd, James Handy, Gary Joseph Thor...",[Joe Johnston]
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,grumpier old men,6.5,92.0,1995,"[Comedy, Romance]",Grumpy Old Men Collection,Warner Bros. Lancaster Gate,United States of America,English,0.000000,"[Daryl Hannah, Sophia Loren, Jack Lemmon, Burg...",[Howard Deutch]
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,waiting to exhale,6.1,34.0,1995,"[Comedy, Drama, Romance]",[],Twentieth Century Fox Film Corporation,United States of America,English,5.090760,"[Lela Rochon, Michael Beach, Dennis Haysbert, ...",[Forest Whitaker]
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,father of the bride part ii,5.7,173.0,1995,[Comedy],Father of the Bride Collection,Sandollar Productions Touchstone Pictures,United States of America,English,0.000000,"[Eugene Levy, Steve Martin, Kate McGregor-Stew...",[Charles Shyer]


## Guardado
<p>Previo a guardar se realiza una seleccion de columnas a utilizar</p>

In [118]:
# Posibles datos a utilizar en el sistema de recomendacion
rec_system = movies_merged[["title","overview","name_genre","tagline","release_year","vote_average"]]

In [119]:
# Columnas a utilizar en el desarrollo de la API
movies_merged_api = movies_merged[["title","revenue","budget","return","vote_average","vote_count","release_date","release_year","actors_names","director_names"]]

* Dataset para el sistema de recomendacion 

In [120]:
rec_system.to_parquet("../datasets/rec_system.parquet")

* Dataset para el desarrollo de la API 

In [121]:
# movies_merged.to_csv(path_or_buf="../datasets/movies_merged.csv")
movies_merged_api.to_parquet("../datasets/movies_merged.parquet")


* Dataset utilizado para el EDA

In [122]:
movies_merged.to_parquet("../datasets/movies_eda.parquet")
# movies_merged.to_csv("../datasets/movies_eda.csv")